# **An Exampe of Using the DGCCA Package - dgcca_pckg (Under Development)**
An example of using the dgcca package to the 'Mushroom Classification' dataset (Source :https://www.kaggle.com/uciml/mushroom-classification) to classify mushrooms as 'edible' or 'poisonous'.

---


**Dataset Description** - Cardiotocograms (CTGs) are a simple and cost accessible option to assess fetal health, allowing doctors to take action to prevent child and maternal mortality. This dataset includes 2126 records of features extracted from Cardiotocogram exams, which are classified into 3 classes: Normal, Suspect, Pathological.

---

**Objective** - Create a multiclass model to classify CTG features into the three fetal health states.


---

**Classification Models Used** -



---




In [17]:
!pip install cca-zoo

In [110]:
#Importing required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import io
from sklearn.model_selection import train_test_split
 
# import dgcca
# !python dgcca.py

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


In [113]:
#uploading GlobalTempratures.csv on Google Colab
from google.colab import files
uploaded = files.upload()

Saving fetal_health.csv to fetal_health (1).csv


In [114]:
#Reading Data
data = pd.read_csv(io.BytesIO(uploaded['fetal_health.csv']))
data.head()

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,mean_value_of_long_term_variability,histogram_width,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency,fetal_health
0,120.0,0.000,0.0,0.000,0.000,0.0,0.0,73.0,0.5,43.0,2.4,64.0,62.0,126.0,2.0,0.0,120.0,137.0,121.0,73.0,1.0,2.0
1,132.0,0.006,0.0,0.006,0.003,0.0,0.0,17.0,2.1,0.0,10.4,130.0,68.0,198.0,6.0,1.0,141.0,136.0,140.0,12.0,0.0,1.0
2,133.0,0.003,0.0,0.008,0.003,0.0,0.0,16.0,2.1,0.0,13.4,130.0,68.0,198.0,5.0,1.0,141.0,135.0,138.0,13.0,0.0,1.0
3,134.0,0.003,0.0,0.008,0.003,0.0,0.0,16.0,2.4,0.0,23.0,117.0,53.0,170.0,11.0,0.0,137.0,134.0,137.0,13.0,1.0,1.0
4,132.0,0.007,0.0,0.008,0.000,0.0,0.0,16.0,2.4,0.0,19.9,117.0,53.0,170.0,9.0,0.0,137.0,136.0,138.0,11.0,1.0,1.0


In [115]:
data.describe()

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,mean_value_of_long_term_variability,histogram_width,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency,fetal_health
count,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.00000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000,2126.000000
mean,133.303857,0.003178,0.009481,0.004366,0.001889,0.000003,0.000159,46.990122,1.332785,9.84666,8.187629,70.445908,93.579492,164.025400,4.068203,0.323612,137.452023,134.610536,138.090310,18.808090,0.320320,1.304327
std,9.840844,0.003866,0.046666,0.002946,0.002960,0.000057,0.000590,17.192814,0.883241,18.39688,5.628247,38.955693,29.560212,17.944183,2.949386,0.706059,16.381289,15.593596,14.466589,28.977636,0.610829,0.614377
min,106.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.000000,0.200000,0.00000,0.000000,3.000000,50.000000,122.000000,0.000000,0.000000,60.000000,73.000000,77.000000,0.000000,-1.000000,1.000000
25%,126.000000,0.000000,0.000000,0.002000,0.000000,0.000000,0.000000,32.000000,0.700000,0.00000,4.600000,37.000000,67.000000,152.000000,2.000000,0.000000,129.000000,125.000000,129.000000,2.000000,0.000000,1.000000
50%,133.000000,0.002000,0.000000,0.004000,0.000000,0.000000,0.000000,49.000000,1.200000,0.00000,7.400000,67.500000,93.000000,162.000000,3.000000,0.000000,139.000000,136.000000,139.000000,7.000000,0.000000,1.000000
75%,140.000000,0.006000,0.003000,0.007000,0.003000,0.000000,0.000000,61.000000,1.700000,11.00000,10.800000,100.000000,120.000000,174.000000,6.000000,0.000000,148.000000,145.000000,148.000000,24.000000,1.000000,1.000000
max,160.000000,0.019000,0.481000,0.015000,0.015000,0.001000,0.005000,87.000000,7.000000,91.00000,50.700000,180.000000,159.000000,238.000000,18.000000,10.000000,187.000000,182.000000,186.000000,269.000000,1.000000,3.000000


In [116]:
data.info() 
#The dataset has no missing values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2126 entries, 0 to 2125
Data columns (total 22 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   baseline value                                          2126 non-null   float64
 1   accelerations                                           2126 non-null   float64
 2   fetal_movement                                          2126 non-null   float64
 3   uterine_contractions                                    2126 non-null   float64
 4   light_decelerations                                     2126 non-null   float64
 5   severe_decelerations                                    2126 non-null   float64
 6   prolongued_decelerations                                2126 non-null   float64
 7   abnormal_short_term_variability                         2126 non-null   float64
 8   mean_value_of_short_term_variability  

In [117]:
# counting no of unique values of each column
for x in data.columns.array:
  print(x, (data[x]).nunique()) 

baseline value 48
accelerations 20
fetal_movement 102
uterine_contractions 16
light_decelerations 16
severe_decelerations 2
prolongued_decelerations 6
abnormal_short_term_variability 75
mean_value_of_short_term_variability 57
percentage_of_time_with_abnormal_long_term_variability 87
mean_value_of_long_term_variability 249
histogram_width 154
histogram_min 109
histogram_max 86
histogram_number_of_peaks 18
histogram_number_of_zeroes 9
histogram_mode 88
histogram_mean 103
histogram_median 95
histogram_variance 133
histogram_tendency 3
fetal_health 3




---



## DGCCA

In [364]:
#importing required libraries -
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim 
from torch.utils.data import BatchSampler, SequentialSampler, RandomSampler
from cca_zoo.objectives import GCCA as GCCA_Loss
from cca_zoo.wrapper import Wrapper as GCCA
#Loss() method takes the outputs of each view's network and solves the generalized CCA eigenproblem

In [365]:
class DNN(nn.Module):
    def __init__(self, layer_size, activation):
        super(DNN, self).__init__()
        layers = []
        self.activation = activation
        
        # Defaults to sigmoid 
        if self.activation == 'relu':
          self.activation_func = nn.RelU()
        elif self.activation == 'tanh':
          self.activation_func = nn.Tanh()
        elif self.activation == 'sigmoid':
          self.activation_func = nn.Sigmoid()
        else:
          self.activation_func = nn.Sigmoid()

        for l_id in range(len(layer_size) - 1):
            if l_id == len(layer_size) - 2: #second last layer
                layers.append(nn.Sequential(
                    nn.BatchNorm1d(num_features=layer_size[l_id], affine=False),
                    nn.Linear(layer_size[l_id], layer_size[l_id + 1]),
                ))
            else: #all other layers
                layers.append(nn.Sequential(
                    nn.Linear(layer_size[l_id], layer_size[l_id + 1]), 
                    self.activation_func,
                    nn.BatchNorm1d(num_features=layer_size[l_id + 1], affine=False),                                     
                ))

        self.layers = nn.ModuleList(layers)

    def forward(self, l):
        for layer in self.layers:
            l = layer(l)
        return l

In [366]:
class DGCCA_architecture(nn.Module): # for three views
    def __init__(self, layer_size1, layer_size2, layer_size3, activation):
        super(DGCCA_architecture, self).__init__()
        self.activation = activation
        self.model1 = DNN(layer_sizes1,activation)
        self.model2 = DNN(layer_sizes2,activation)
        self.model3 = DNN(layer_sizes2,activation)

    def forward(self, x1, x2, x3):
        output1 = self.model1(x1)
        output2 = self.model2(x2)
        output3 = self.model3(x3)

        return output1, output2, output3

In [377]:
class DGCCA(nn.Module):
  def __init__(self, architecture, learning_rate, epoch_num, batch_size, reg_par, out_size:int):
        super(DGCCA, self).__init__()
        self.arch = architecture
        self.learning_rate = learning_rate
        self.reg_par = reg_par
        # Stochastic Gradient Descent used as optimizer like in the research paper
        self.optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=reg_par)
        self.epoch_num = epoch_num
        self.batch_size = batch_size
        self.out_size = out_size
        # The GCCA loss function
        self.loss_function = GCCA_Loss(self.out_size , self.reg_par).loss

  def _get_outputs(self, x1, x2, x3):
        with torch.no_grad():
            self.arch.eval()
            data_size = x1.size(0)
            batch_idxs = list(BatchSampler(SequentialSampler(
                range(data_size)), batch_size=self.batch_size, drop_last=False))
            losses = []
            outputs1 = []
            outputs2 = []
            outputs3 = []
            for x in batch_idxs:
                batch_x1 = x1[x, :]
                batch_x2 = x2[x, :]
                batch_x3 = x3[x, :]
                o1, o2, o3 = self.arch(batch_x1, batch_x2, batch_x3)
                outputs1.append(o1)
                outputs2.append(o2)
                outputs3.append(o3)
                loss = self.loss_function(o1, o2, o3)
                losses.append(loss.item())
        outputs = [torch.cat(outputs1, dim=0).numpy(),
                   torch.cat(outputs2, dim=0).numpy(),
                   torch.cat(outputs3, dim=0).numpy()]
        return losses, outputs

  def test(self, x1, x2, x3):
        with torch.no_grad():
            losses, outputs = self._get_outputs(x1, x2, x3)
            return np.mean(losses), outputs

  def fit(self, train_x1, train_x2, train_x3, test_x1, test_x2, test_x3):
      data_size = train_x1.size(0)
      train_losses = []
      for epoch in range(self.epoch_num):
          self.arch.train()
          batch_idxs = list(BatchSampler(RandomSampler(
              range(data_size)), batch_size=self.batch_size, drop_last=False))
          for x in batch_idxs:
              self.optimizer.zero_grad()           
              batch_x1 = train_x1[x, :]
              batch_x2 = train_x2[x, :]
              batch_x3 = train_x3[x, :] 
              o1, o2, o3 = self.arch(batch_x1, batch_x2, batch_x3)
              loss = self.loss_function(o1, o2, o3)
              train_losses.append(loss.data)
              loss.backward()
              self.optimizer.step()

          # training the gcca model
          _, outputs = self._get_outputs(train_x1, train_x2, train_x3)
          GCCA_obj = GCCA(self.out_size, method="gcca")
          GCCA_obj.fit(outputs[0], outputs[1], outputs[2],params=None) #function from cca-zoo

          loss = self.test(test_x1, test_x2, test_x3)
      print("Fitted Model to Data")

## Data Preprocessing

In [368]:
#Features for the model
X = data.iloc[:,1:21]

In [369]:
# converting to tensor
X = torch.from_numpy(X.to_numpy())

In [385]:
#Splitting views in testing and training set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=0)

In [386]:
#Creating different views
X1_train = X_train[0:2125][0:7]
X2_train = X_train[0:2125][7:14]
X3_train = X_train[0:2125][14:21]
X1_test = X_test[0:2125][0:7]
X2_test = X_test[0:2125][7:14]
X3_test = X_test[0:2125][14:21]

In [387]:
X1_train =torch.transpose(X1_train, 0, 1).float()
X2_train =torch.transpose(X2_train, 0, 1).float()
X3_train =torch.transpose(X3_train, 0, 1).float()
X1_test =torch.transpose(X1_test, 0, 1).float()
X2_test =torch.transpose(X2_test, 0, 1).float()
X3_test =torch.transpose(X3_test, 0, 1).float()

## Applying DGCCA

In [388]:
in_size_X1 = 7
in_size_X2 = 7
in_size_X3 = 7
out_size = 2

layer_sizes1 = [in_size_X1, 50, 50, out_size]
layer_sizes2 = [in_size_X1, 50, 50, out_size]
layer_sizes3 = [in_size_X1, 50, 50, out_size]

model = DGCCA_architecture(layer_sizes1, layer_sizes2, layer_sizes3, "sigmoid")

learning_rate = 1e-3
epoch_num = 100
batch_size = 80
reg_par = 1e-5

#DGCCA(self, architecture, learning_rate, epoch_num, batch_size, reg_par, out_size:int)
algo = DGCCA(model, learning_rate, epoch_num, batch_size, reg_par, out_size)
algo.fit(X1_train, X2_train, X3_train, X1_test, X2_test, X3_test)

more than 2 views therefore switched to generalized
more than 2 views therefore switched to generalized
more than 2 views therefore switched to generalized
more than 2 views therefore switched to generalized
more than 2 views therefore switched to generalized
more than 2 views therefore switched to generalized
more than 2 views therefore switched to generalized
more than 2 views therefore switched to generalized
more than 2 views therefore switched to generalized
more than 2 views therefore switched to generalized
more than 2 views therefore switched to generalized
more than 2 views therefore switched to generalized
more than 2 views therefore switched to generalized
more than 2 views therefore switched to generalized
more than 2 views therefore switched to generalized
more than 2 views therefore switched to generalized
more than 2 views therefore switched to generalized
more than 2 views therefore switched to generalized
more than 2 views therefore switched to generalized
more than 2 

In [389]:
loss, outputs = algo.test(torch.cat([X1_train, X1_test], dim=0), torch.cat([X2_train, X2_test], dim=0),torch.cat([X3_train, X3_test], dim=0))

In [ ]:
#Combining views

## Creating ML Models

1. Logistic Regression

In [380]:
from sklearn.linear_model import LogisticRegression

In [393]:
# lg_model = LogisticRegression()
# lg_model.fit(X1_train, Y_train[0:3000][0:7])

ValueError: ignored



---

**Conclusion** - 


---


---


